## Simulate data CRDM

This script is used to generate simulated data for the CRDM task. Then we can run parameter_recovery to estimate those parameters from the simulated data


In [ ]:
# Built-in/Generic Imports
import os

# Libs
import numpy as np
import pandas as pd
from scipy.stats import bernoulli

In [35]:

### Defining the grids, and functions to make sets as dataframes: choice_set,param_set

def set_grids(nb_alpha=10,nb_beta=10,nb_gamma=10):
    grid_design = {
        # probability of safe bet
        'crdm_sure_p': [1.0],
        # probability of reward (lottery winning probability)
        'crdm_lott_p': [0.13, 0.25, 0.38, 0.50, 0.75],
        # safe reward: $5
        'crdm_sure_amt': [5],
        # reward amount set to vary according to experiment
        'crdm_lott_amt': [5, 8, 20, 40, 50],
        # ambiguity levels
        'crdm_amb_lev':[0.0, 0.24, 0.50, 0.74]
    }
    grid_param = {
        'alpha_gt': np.linspace(0, 3, nb_alpha+1)[1:],
        # 'beta': np.linspace(-3, 3, 11),
        # setting beta_max = 1.3 to avoid getting negative SV when generating new choice set
        'beta_gt': np.linspace(-3, 3, nb_beta),
        'gamma_gt': np.linspace(0, 5, nb_gamma+1)[1:]
    }
    return grid_design,grid_param

def insert_var(grid=[],var_nb=0,list_var=[]):
    if not var_nb:
        grid = list_var
    elif var_nb==1:
        grid = [[g]+[i] for g in grid for i in list_var]
    else:
        grid = [g+[i] for g in grid for i in list_var]
    return grid

def make_grid(grid_values):
    grid = []
    labels = grid_values.keys()
    for var_nb,row in enumerate(grid_values.items()):
        grid = insert_var(grid=grid,var_nb=var_nb,list_var=row[1])
    grid_df = pd.DataFrame(grid,columns=labels)
    return grid_df


In [36]:

def simulate_response(design,PARAM_TRUE):
    # Calculate the probability to choose a variable option
    pn, pr, vn, vr, ambig = (design['crdm_sure_p'], design['crdm_lott_p'],design['crdm_sure_amt'], design['crdm_lott_amt'],design['crdm_amb_lev'])
    alpha, beta, gamma = PARAM_TRUE['alpha_gt'], PARAM_TRUE['beta_gt'], PARAM_TRUE['gamma_gt']

    SV_null = (vn**alpha) * pn
    SV_reward = (vr**alpha) * (pr - beta * ambig / 2)
    p_obs = 1. / (1 + np.exp(-gamma * (SV_reward - SV_null)))

    # Randomly sample a binary choice response from Bernoulli distribution
    return float(bernoulli.rvs(p_obs))

def generate_response(design_df,PARAM_TRUE):
    response= []
    df = pd.DataFrame([])
    for index, design in design_df.iterrows():
        response += [simulate_response(design,PARAM_TRUE)]
    df['crdm_choice'] = response
    df['crdm_trial_type'] = 'task'
    df['crdm_confkey'] = 1.0
    return df


In [37]:
# can set resolution (number of parameter) for kappa and gamma
grid_design,grid_param = set_grids(nb_alpha=5,nb_beta=5,nb_gamma=5)

design_df = make_grid(grid_design)
fn = os.path.join('simul','design_set.csv')
print('Saving design to : {}'.format(fn))
design_df.to_csv(fn)

param_df = make_grid(grid_param)
fn = os.path.join('simul','ground_truth.csv')
print('Saving ground truth parameters to : {}'.format(fn))
param_df.to_csv(fn)

for index, PARAM_TRUE in param_df.iterrows():
    response_df = generate_response(design_df,PARAM_TRUE)
    subj_dir = os.path.join('simul','split','p{0:04d}'.format(index),'crdm')
    if not os.path.exists(subj_dir):
        os.makedirs(subj_dir)
    fn = os.path.join(subj_dir,'p{0:04d}_crdm.csv'.format(index))
    print('Saving response to : {}'.format(fn))
    df = pd.concat([design_df,response_df],axis=1)
    df.to_csv(fn)


Saving design to : simul/design_set.csv
Saving ground truth parameters to : simul/ground_truth.csv
Saving response to : simul/split/p0000/cdd/p0000_cdd.csv
Saving response to : simul/split/p0001/cdd/p0001_cdd.csv
Saving response to : simul/split/p0002/cdd/p0002_cdd.csv
Saving response to : simul/split/p0003/cdd/p0003_cdd.csv
Saving response to : simul/split/p0004/cdd/p0004_cdd.csv
Saving response to : simul/split/p0005/cdd/p0005_cdd.csv
Saving response to : simul/split/p0006/cdd/p0006_cdd.csv
Saving response to : simul/split/p0007/cdd/p0007_cdd.csv
Saving response to : simul/split/p0008/cdd/p0008_cdd.csv
Saving response to : simul/split/p0009/cdd/p0009_cdd.csv
Saving response to : simul/split/p0010/cdd/p0010_cdd.csv
Saving response to : simul/split/p0011/cdd/p0011_cdd.csv
Saving response to : simul/split/p0012/cdd/p0012_cdd.csv
Saving response to : simul/split/p0013/cdd/p0013_cdd.csv
Saving response to : simul/split/p0014/cdd/p0014_cdd.csv
Saving response to : simul/split/p0015/cdd/p00